In [165]:
from bs4 import BeautifulSoup 
import requests 
import csv
import pandas as pd
import spacy

import re
from nltk.stem.porter import PorterStemmer

In [5]:
reviews = []
ratings = []

In [186]:
pages = list(range(1,100))
for page in pages:
    req = requests.get("https://www.flipkart.com/samsung-galaxy-f22-denim-blue-128-gb/product-reviews/itm30c755943bea0?pid=MOBG43UGRSMJXTMH&lid=LSTMOBG43UGRSMJXTMH3SYSZ3&marketplace=FLIPKART&page={}".format(page)).text  # URL of the website which you want to scrape
    soup = BeautifulSoup(req,'html.parser')

In [187]:
rating = soup.find_all('div',attrs={'class':'_3LWZlK _1BLPMq'})
for i in range(len(rating)):
    ratings.append(rating[i].text)
    len(ratings)
    
review = soup.find_all('div',attrs={'class':'t-ZTKy'})
for i in range(len(review)):
    reviews.append(review[i].text)
    len(reviews)

In [188]:
print(len(ratings))

754


In [189]:
len(reviews)

840

In [198]:
df = {'ratings':ratings[:700],'reviews':reviews[:700]}
df_1 = pd.DataFrame(df)
df_1.head(50)

,ratings,reviews
0,4,Very bad experience with Samsung galaxy I thou...
1,5,Beast mobile Their is issueREAD MORE
2,5,Very good quality phone Super BatteryREAD MORE
3,5,Speaker loudness is Ok okzDisplay 8 /10Perform...
4,4,Good Quality batteryREAD MORE
5,4,Value for moneyREAD MORE
6,5,"They should provide ear phones, and charger's ..."
7,4,Best product in 13499 priceREAD MORE
8,4,"Nice product, best phone in this priceREAD MORE"
9,5,badREAD MORE


# Data Preparation
### Step 1: Prepare a pipline for text preprocessing

In [199]:
# Emotion Lexicon - Affin
affin=pd.read_csv('Afinn.csv',sep=',',encoding='Latin-1')
affin

,word,value
0,abandon,-2
1,abandoned,-2
2,abandons,-2
3,abducted,-2
4,abduction,-2
...,...,...
2472,yucky,-2
2473,yummy,3
2474,zealot,-2
2475,zealots,-2


In [200]:
affinity_scores=affin.set_index('word')['value'].to_dict()
affinity_scores

{'abandon': -2,
 'abandoned': -2,
 'abandons': -2,
 'abducted': -2,
 'abduction': -2,
 'abductions': -2,
 'abhor': -3,
 'abhorred': -3,
 'abhorrent': -3,
 'abhors': -3,
 'abilities': 2,
 'ability': 2,
 'aboard': 1,
 'absentee': -1,
 'absentees': -1,
 'absolve': 2,
 'absolved': 2,
 'absolves': 2,
 'absolving': 2,
 'absorbed': 1,
 'abuse': -3,
 'abused': -3,
 'abuses': -3,
 'abusive': -3,
 'accept': 1,
 'accepted': 1,
 'accepting': 1,
 'accepts': 1,
 'accident': -2,
 'accidental': -2,
 'accidentally': -2,
 'accidents': -2,
 'accomplish': 2,
 'accomplished': 2,
 'accomplishes': 2,
 'accusation': -2,
 'accusations': -2,
 'accuse': -2,
 'accused': -2,
 'accuses': -2,
 'accusing': -2,
 'ache': -2,
 'achievable': 1,
 'aching': -2,
 'acquit': 2,
 'acquits': 2,
 'acquitted': 2,
 'acquitting': 2,
 'acrimonious': -3,
 'active': 1,
 'adequate': 1,
 'admire': 3,
 'admired': 3,
 'admires': 3,
 'admiring': 3,
 'admit': -1,
 'admits': -1,
 'admitted': -1,
 'admonish': -2,
 'admonished': -2,
 'adopt': 

In [201]:
# Custom function: score each word in a sentence in lemmatised form, but calculate the score for the whole original sentence
nlp=spacy.load('en_core_web_sm')
sentiment_lexicon=affinity_scores

def calculate_sentiment(text:str=None):
    sent_score=0
    if text:
        sentence=nlp(text)
        for word in sentence:
            sent_score+=sentiment_lexicon.get(word.lemma_,0)
    return sent_score

In [202]:
# manual testing
calculate_sentiment(text='great')

3

In [203]:
# Calculating sentiment value for each sentence
df_1['sentiment_value']=df_1['reviews'].apply(calculate_sentiment)
df_1['sentiment_value']

0      -3
1       0
2       3
3       0
4       3
       ..
695     5
696    23
697     4
698     0
699     6
Name: sentiment_value, Length: 700, dtype: int64

In [204]:
# how many words are there in a sentence?
df_1['word_count']=df_1['reviews'].str.split().apply(len)
df_1['word_count']

0      59
1       6
2       7
3      18
4       4
       ..
695    29
696    81
697     9
698     5
699    16
Name: word_count, Length: 700, dtype: int64

In [205]:
df_1.sort_values(by='sentiment_value')

,ratings,reviews,sentiment_value,word_count
0,4,Very bad experience with Samsung galaxy I thou...,-3,59
10,5,Very bad experience with Samsung galaxy I thou...,-3,59
20,4,Very bad experience with Samsung galaxy I thou...,-3,59
208,5,Average under this priceREAD MORE,0,5
431,4,GoodREAD MORE,0,2
...,...,...,...,...
256,4,I recently purchased this newly launched Samsu...,23,81
266,3,I recently purchased this newly launched Samsu...,23,81
446,3,I recently purchased this newly launched Samsu...,23,81
286,5,I recently purchased this newly launched Samsu...,23,81


In [207]:
# negative sentiment score of the whole review
df_1[df_1['sentiment_value']<=0]

,ratings,reviews,sentiment_value,word_count
0,4,Very bad experience with Samsung galaxy I thou...,-3,59
1,5,Beast mobile Their is issueREAD MORE,0,6
3,5,Speaker loudness is Ok okzDisplay 8 /10Perform...,0,18
5,4,Value for moneyREAD MORE,0,4
6,5,"They should provide ear phones, and charger's ...",0,13
...,...,...,...,...
688,4,Average under this priceREAD MORE,0,5
691,5,GoodREAD MORE,0,2
692,4,NiceREAD MORE,0,2
693,5,"Call volume is less, the weight is moreREAD MORE",0,9


In [208]:
# positive sentiment score of the whole review
df_1[df_1['sentiment_value']>0]

,ratings,reviews,sentiment_value,word_count
2,5,Very good quality phone Super BatteryREAD MORE,3,7
4,4,Good Quality batteryREAD MORE,3,4
7,4,Best product in 13499 priceREAD MORE,3,6
8,4,"Nice product, best phone in this priceREAD MORE",6,8
12,4,Very good quality phone Super BatteryREAD MORE,3,7
...,...,...,...,...
694,5,Best Phone Ever....!!!READ MORE,3,4
695,5,The phone is very good but whene we charge it ...,5,29
696,5,I recently purchased this newly launched Samsu...,23,81
697,4,"Yes as given in pic,a very good phoneREAD MORE",4,9
